In [ ]:
# Importar librerías
import cornac
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF
from cornac.metrics import RMSE, MAE, Precision, NDCG

# Cargar los datos
reader = Reader()
ml_data = reader.read(fpath='datasets/ml-100k/u.data', fmt='UIRT', sep='\t')

# Crear método de evaluación (split)
eval_method = RatioSplit(
    data=ml_data,
    test_size=0.2,
    rating_threshold=0.0,  # considera todos los ratings
    seed=42,
    verbose=True
)

In [ ]:
# Modelo 1: Matrix Factorization (MF)
model_mf = MF(k=10, max_iter=50, learning_rate=0.01, verbose=True)

mf_results = eval_method.evaluate(model_mf, metrics=[RMSE(), MAE(), Precision(k=5), NDCG(k=5)], user_based=True)

for metric, value in mf_results[0].metric_avg_results.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Modelo 2: PMF
model_pmf = PMF(k=10, max_iter=50, learning_rate=0.01, verbose=True)

pmf_results = eval_method.evaluate(model_pmf, metrics=[RMSE(), MAE(), Precision(k=5), NDCG(k=5)], user_based=True)

for metric, value in pmf_results[0].metric_avg_results.items():
    print(f"{metric}: {value:.4f}")

| Modelo | RMSE   | MAE    | Precision@5  | NDCG@5  | TRAIN(S) | TEST(S)  |
|--------|--------|--------|--------------|---------|----------|----------|
| MF     | 0.9239 | 0.7507 | 0.0732       | 0.0762  | 0.1583   | 0.7775   |
| PMF    | 1.0140 | 0.8266 | 0.0506       | 0.0492  | 0.7237   | 0.5035   |



#### **Qué mide cada métrica?**

| Métrica          | Qué mide exactamente                                                                                | Valor ideal                                 |
| ---------------- | --------------------------------------------------------------------------------------------------- | ------------------------------------------- |
| **RMSE**         | *Root Mean Squared Error*: error promedio cuadrático. Penaliza fuerte los errores grandes.          | Lo más bajo posible                         |
| **MAE**          | *Mean Absolute Error*: promedio del error absoluto entre rating real y predicho.                    | Lo más bajo posible                         |
| **Precision\@5** | De los 5 ítems recomendados a cada usuario, ¿qué proporción fueron relevantes?                      | Más alto = mejor                            |
| **NDCG\@5**      | *Normalized Discounted Cumulative Gain*: similar a Precision, pero considera el orden de los ítems. | Más alto = mejor                            |
| **TRAIN(S)**     | Tiempo de entrenamiento del modelo, en segundos.                                                    | Más bajo es mejor (si hay mucha diferencia) |
| **TEST(S)**      | Tiempo de evaluación del modelo (predicción y métricas).                                            | Más bajo es mejor (si afecta escalabilidad) |

---

#### **Interpretación por métrica**

**RMSE y MAE:**

* MF tiene menor error (0.92 vs 1.01 y 0.75 vs 0.82).
* Esto significa que MF predice mejor los ratings reales, con menos desviación promedio que PMF.

**Precision\@5 y NDCG\@5:**

* MF también supera claramente a PMF:

  * Precision\@5: MF logra \~7.3% de acierto, mientras PMF solo \~5.0%.
  * NDCG\@5: MF \~7.6%, PMF \~4.9%.
* Esto indica que MF recomienda ítems más relevantes y en mejor orden que PMF.

> Aunque parezcan valores bajos, es normal en datasets grandes y dispersos como MovieLens 100k.

##### **Conclusiones generales**

1. MF es un modelo más preciso y eficiente que PMF en este dataset.
2. PMF puede ser útil en escenarios probabilísticos, pero en este caso no ofrece ventajas claras.
3. Precision\@k y NDCG son métricas clave cuando lo que te importa no es el número exacto del rating, sino si recomendás lo que el usuario va a disfrutar.
